In [1]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 85.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.4 MB/s eta 0:00:00:00:01


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer

# Load your data
df = pd.read_csv('/kaggle/input/dynahate/DynaHate.csv')
df.reset_index(drop=True, inplace=True)
# Encode the labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])


# Split the data and reset the indices
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)
train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
test_labels.reset_index(drop=True, inplace=True)



In [4]:
df.size

534872

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, PeftType
import torch


model_name="Qwen/Qwen2-1.5B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Tokenize the text
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)


2024-07-08 14:45:50.454290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 14:45:50.454437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 14:45:50.589065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HateSpeechDataset(train_encodings, train_labels)
test_dataset = HateSpeechDataset(test_encodings, test_labels)


In [7]:
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id

# Define LoRA configuration
lora_config = LoraConfig(
    peft_type=PeftType.LORA,
    task_type="SEQ_CLS",
    r=2,  # rank of the low-rank adaptation matrices
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=['k_proj', 'v_proj'],
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
    logging_steps=200,
    eval_strategy="steps",
    gradient_accumulation_steps=8,
    fp16=True,
    save_strategy="no"
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)



In [10]:
from sklearn.metrics import classification_report
for epoch in range(3):
    trainer.train()
    
    #Evaluate the model
    preds_output = trainer.predict(test_dataset)
    preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

    # Generate classification report
    print(classification_report(test_labels, preds, target_names=le.classes_))

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss,Validation Loss
200,1.436400,1.046006
400,0.856100,0.692631


              precision    recall  f1-score   support

        hate       0.72      0.77      0.74      4376
     nothate       0.71      0.66      0.68      3853

    accuracy                           0.72      8229
   macro avg       0.72      0.71      0.71      8229
weighted avg       0.72      0.72      0.71      8229



Step,Training Loss,Validation Loss
200,0.557400,0.518949
400,0.490500,0.443459


              precision    recall  f1-score   support

        hate       0.81      0.83      0.82      4376
     nothate       0.80      0.78      0.79      3853

    accuracy                           0.81      8229
   macro avg       0.81      0.81      0.81      8229
weighted avg       0.81      0.81      0.81      8229



Step,Training Loss,Validation Loss
200,0.416800,0.394313
400,0.394800,0.376943


              precision    recall  f1-score   support

        hate       0.86      0.83      0.85      4376
     nothate       0.82      0.84      0.83      3853

    accuracy                           0.84      8229
   macro avg       0.84      0.84      0.84      8229
weighted avg       0.84      0.84      0.84      8229



In [ ]:
from sklearn.metrics import classification_report

# Get predictions
preds_output = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

# Generate classification report
print(classification_report(test_labels, preds, target_names=le.classes_))


In [ ]:
model.push_to_hub("Tapan101/hateQwen", check_pr=True)
tokenizer.push_to_hub("Tapan101/hateQwen", check_pr=True)